In [66]:
from mne.filter import filter_data
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pyedflib
import numpy as np
from datetime import timedelta
from scipy.signal import find_peaks_cwt, decimate
from scipy.stats import spearmanr
import time
from entropy import shannon_entropy, sample_entropy

from os import listdir
    #method listdir() returns a list containing the names of the entries in the directory given by path.
from os.path import isfile, join
    #returns true if file in path is an existing regular file

In [4]:
## !! set correct working directory
#import os
# path="/Volumes/Projects_div3/ADBS"
path = "Y:/ADBS"
#os.chdir(path)
#os.getcwd() #check

In [6]:
# prepare ESM export from server
esm = df = pd.read_stata(path+'/esmDataPilot/PRDB_20181025T115653/SANPAR_BE.dta',convert_categoricals = False)
esm = esm[['subjno', 'mood_well', 'mood_down', 'mood_fright', 'mood_tense', 'phy_sleepy', 'phy_tired',
       'mood_cheerf', 'mood_relax', 'thou_concent', 'pat_hallu', 'loc_where',
       'soc_who', 'soc_who02', 'soc_who03', 'act_what', 'act_what02',
       'act_what03', 'act_norpob', 'sanpar_been', 'sanpar_stil',
       'sanpar_spreken', 'sanpar_lopen', 'sanpar_tremor', 'sanpar_traag',
       'sanpar_stijf', 'sanpar_spann', 'sanpar_beweeg', 'sanpar_onoff',
       'sanpar_medic', 'beep_disturb', '_datetime', '_datetime_e', 'dayno_n', 'beepno_n']]
esm['duration'] = esm['_datetime_e']-esm['_datetime']

mapNames={}
for i in range(20):
    mapNames[9009989+i]=110001+i

esm['castorID'] = [mapNames[e] for e in esm['subjno']]

C:\ProgramData\Miniconda3\envs\p3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Miniconda3\envs\p3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [7]:
# create list of files per L/R/chest from directory (mypath)

leftSensors = ['13797','13799','13794']
rightSensors = ['13805','13801','13793']
chestSensors = ['13804','13792','13803']

featureWindowLength=60
windowLength=60
esmWindowLength=15

mypath = 'C:/data/raw/MOX/110005/'
bdffiles = [f for f in listdir(mypath) if isfile(join(mypath,f)) and f[0]!='_' and f[-3:] =='edf']
#bdffiles are the files in mypath, not directories

leftFiles = []
rightFiles = []
chestFiles = []

for f in bdffiles:
    if f[0:5] in leftSensors:
        leftFiles.append(mypath + f)
    elif f[0:5] in rightSensors:
        rightFiles.append(mypath + f)
    elif f[0:5] in chestSensors:
        chestFiles.append(mypath + f)

leftFiles=sorted(leftFiles)
rightFiles=sorted(rightFiles)
chestFiles=sorted(chestFiles)

In [69]:
testL,testR,testC = extractAllSensors(leftFiles,rightFiles,chestFiles)

C:/data/raw/MOX/110005/13799_20181002_060804.edf


C:\ProgramData\Miniconda3\envs\p3\lib\site-packages\scipy\signal\_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]
C:\ProgramData\Miniconda3\envs\p3\lib\site-packages\scipy\signal\signaltools.py:3463: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return y[sl]


C:/data/raw/MOX/110005/13799_20181003_070253.edf
C:/data/raw/MOX/110005/13799_20181003_070253.edf is broken
C:/data/raw/MOX/110005/13799_20181003_070256.edf
C:/data/raw/MOX/110005/13799_20181004_000000.edf
C:/data/raw/MOX/110005/13799_20181004_083502.edf
C:/data/raw/MOX/110005/13799_20181005_074601.edf
C:/data/raw/MOX/110005/13799_20181006_000000.edf
C:/data/raw/MOX/110005/13799_20181006_084256.edf
C:/data/raw/MOX/110005/13799_20181007_082759.edf
C:/data/raw/MOX/110005/13799_20181008_083214.edf
C:/data/raw/MOX/110005/13799_20181009_081701.edf
C:/data/raw/MOX/110005/13799_20181010_071928.edf
C:/data/raw/MOX/110005/13799_20181011_000000.edf
C:/data/raw/MOX/110005/13799_20181011_075902.edf
C:/data/raw/MOX/110005/13799_20181012_080146.edf
C:/data/raw/MOX/110005/13799_20181013_000000.edf
C:/data/raw/MOX/110005/13799_20181013_084619.edf
C:/data/raw/MOX/110005/13799_20181014_000000.edf
C:/data/raw/MOX/110005/13799_20181014_082141.edf
C:/data/raw/MOX/110005/13799_20181015_000000.edf
C:/data/ra

In [70]:
def extractAllSensors(leftFiles,rightFiles,chestFiles,featureWindowLength=60):
    # Read in the three list of files
    #Process leftWristData
    leftWristDF=[]
    rightWristDF=[]
    chestDF=[]
    accelerometerChannel=[0,1,2] # assuming acc xyz 0-1-2 is
    files = [leftFiles, rightFiles, chestFiles]
    dfs = [leftWristDF,rightWristDF, chestDF]
    identifiers = ['l', 'r', 'c']
    
    for i, f in enumerate(files):
        for file in f:
            print(file)
            try:
                labels, startTime, data, sr = readData(file) ## as input instead: leftFiles
                data = decimate(data,2,axis=1)
                sr = int(sr /2)
                if data.shape[1]<sr * featureWindowLength:
                    raise ValueError('File too short to proceed.')
        #Could be left wrist specific or general
            except:
                print('%s is broken' % file)
                continue

    
    
            data[:,accelerometerChannel] = (data[:,accelerometerChannel].T - np.mean(data[:,accelerometerChannel].T,axis=0)).T
            data[:,accelerometerChannel] = filter_data(data[:,accelerometerChannel],sr,0,3,method='iir',verbose='WARNING')
            alignedTimes, features, labels = extractFeatures(data, startTime, sr, featureWindowLength)
            labels=[l + identifiers[i] for l in labels]
            
            if dfs[i] is None:
                dfs[i]=pd.DataFrame(features.T,columns=labels,index=alignedTimes)
            else:
                dfs[i].append(pd.DataFrame(features.T,columns=labels,index=alignedTimes))
    
 
    leftWristDF = pd.concat((leftWristDF[:]))
    rightWristDF = pd.concat((rightWristDF[:]))
    chestDF = pd.concat((chestDF[:]))
    
    return leftWristDF,rightWristDF,chestDF    

In [9]:
def readData(filename): 
    #Extract data
    f = pyedflib.EdfReader(filename)
    sr = f.getSampleFrequencies()[0]
    n = f.signals_in_file
    signal_labels = f.getSignalLabels()
    sigbufs = np.zeros((n, f.getNSamples()[0]))
    for i in np.arange(n):
        sigbufs[i, :] = f.readSignal(i)    
    #Get starting time
    startingTime=f.getStartdatetime() #needs to be tested
    #startingTime=filename[-19:-4]
    #startingTime=pd.to_datetime(startingTime, format='%Y%m%d_%H%M%S', errors='ignore')
    #print(startingTime)
    return signal_labels, startingTime, sigbufs, sr

In [11]:
def extractFeatures(data, startTime, sr, windowLength=60):
    #Filter data between 4 and 8 Hz
    #filtData = filter_data(data, sr, 4,8)

    #Extract some sort of feature for all windows and corresponding time stamps
    numSamples=data.shape[1]
    # Getting number and names of features
    tremorNames, _ = tremorFeatures(data[:,:windowLength*sr], sr)
    bradyNames, _ = bradykinesiaFeatures(data[:,:windowLength*sr], sr)
    
    numWindows = int(np.floor((numSamples/(windowLength*sr))))
    features=np.zeros((len(tremorNames) + len(bradyNames),numWindows))
    alignedTimes=[]
    for i in range(0,numWindows):
        win = i * windowLength * sr
        #Average power per channel
        #features[:,i]=np.mean(filtData[:,win:win+windowLength*sr]**2,axis=1)
        
        _, features[:len(tremorNames),i] = tremorFeatures(data[:,win:win+windowLength*sr],sr,windowLength=windowLength)
        _, features[len(tremorNames):,i] = bradykinesiaFeatures(data[:,win:win+windowLength*sr],sr,windowLength=windowLength)
        # Add the power between 3.6 and 9.4
        #Timestamp at beginning of each window
        alignedTimes.append(startTime + pd.Timedelta('%d s ' % (i * windowLength)))
    return alignedTimes, features, tremorNames + bradyNames

In [16]:
def tremorFeatures(windowData,sr,windowLength=60):
    gyroChannel={'X':3,'Y':4,'Z':5} # gyro is xyz 3-4-5
    if windowData.shape[1]!=sr*windowLength:
        print(windowData.shape)
    freq = np.fft.rfftfreq(windowLength*sr, d=1./sr)
    selected=np.logical_and(freq>3.5,freq<7.5)
    features=[]
    featureNames=[]
    for ch in gyroChannel.keys():
        spec = np.log(np.sum(np.abs(np.fft.rfft(windowData[gyroChannel[ch],:]))[selected]))
        features.append(spec)
        featureNames.append('BandPower' + ch)
    return featureNames, features

In [53]:
import time

In [60]:
def bradykinesiaFeatures(windowData,sr,windowLength=60):
    features=[]
    featureNames=[]
    accelerometerChannel={'X':0,'Y':1,'Z':2} # assuming acc xyz 0-1-2 is
    
    
    #windowData = (windowData.T - np.mean(windowData.T,axis=0)).T
    #t=time.time()
    #windowData = filter_data(windowData,sr,0,3,method='iir',verbose='WARNING')
    #print('Filter takes ' + str(time.time()-t))
    # lowpass filter signal <3 Hz
    
    # Features: (Patel et al IEEE 2009)
    # rms
    # range
    # entropy
    # normalized cross-correlation value and time point
    # dominat frequency and ratio between dominant and rest
    freq = np.fft.rfftfreq(windowLength*sr, d=1./sr)
    
    for ch in accelerometerChannel.keys():
        #t = time.time()
        #ent = shannon_entropy(windowData[accelerometerChannel[ch],:])
        #features.append(ent)
        #featureNames.append('Entropy' + ch)
        #print('Entropy takes ' + str(time.time()-t))
        
        #t = time.time()
        spec = np.abs(np.fft.rfft(windowData[accelerometerChannel[ch],:]))
        domFreq = freq[np.argmax(spec)]
        features.append(domFreq)
        featureNames.append('DomFreq' + ch)
        
        
        domEnergyRatio = np.max(spec) / np.sum(spec)
        features.append(domEnergyRatio)
        featureNames.append('DomEnergyRatio' + ch)
        
        #print('FFT takes ' + str(time.time()-t))
        #t=time.time()
        rms = np.sqrt(np.mean(windowData[accelerometerChannel[ch],:]**2))
        features.append(rms)
        featureNames.append('RMS' + ch)
        #print('rms takes ' + str(time.time()-t))
        
        #t = time.time()
        ampRange = np.max(windowData[accelerometerChannel[ch],:]) - np.min(windowData[accelerometerChannel[ch],:])
        features.append(ampRange)
        featureNames.append('AmpRange' + ch)
        #print('range takes ' + str(time.time()-t))
    
    #t=time.time()
    cCMax=[]
    cCLocs=[]
    for i, ch1 in enumerate(accelerometerChannel.keys()):
        for j,ch2 in enumerate(list(accelerometerChannel.keys())[i+1:]):
            crossCorr = np.correlate(windowData[accelerometerChannel[ch1],:],windowData[accelerometerChannel[ch2],:],'same')
            crossCorr = crossCorr/(np.std(windowData[accelerometerChannel[ch1],:]) * np.std(windowData[accelerometerChannel[ch2],:]))
            
            cCMax.append(np.max(crossCorr))
            cCLocs.append(np.argmax(crossCorr))
    features.append(np.max(cCMax))
    featureNames.append('MaxCC')
    features.append(cCLocs[np.argmax(cCMax)])
    featureNames.append('MaxCCLoc')
    #print('cross-corr takes ' + str(time.time()-t))
        #peaks=find_peaks_cwt(windowData[accelerometerChannel[ch],:],np.arange(1,10))
        #peaks=[1,2,3]
        #features.append(len(peaks))
        #featureNames.append('#Movements' + ch)
        
    #features.append(np.max(windowData[0:3,:]))
    #featureNames.append('MaxMovement')
    return featureNames, features

In [45]:
accelerometerChannel={'X':0,'Y':1,'Z':2}
print(list(accelerometerChannel.keys()))

['X', 'Y', 'Z']


In [ ]:
import sys
sys.path.append('./')
import entropy

In [ ]:
import os
print(os.getcwd())

In [14]:


a = shannon_entropy([0.3,0.3,0.3])
print(a)

-0.0


In [ ]:
testL.keys()

In [ ]:
esm.keys()

In [ ]:
def alignFeaturesESM(listOfDF,esmFrame,esmColumns,esmWindowLength=15):
    
    combinedColumns=esmColumns
    for featureFrame in listOfDF:
        combinedColumns= combinedColumns + featureFrame.keys().tolist()  
    esmFeatures=pd.DataFrame(columns=combinedColumns) # Create new empty dataframe with feature and esm columns

    hop=np.mean(np.diff(listOfDF[0].index))
    for beep in range(esmFrame.shape[0]): #Loop through all the ESM Beeps
        beepTime=esmFrame['_datetime'].iloc[beep] # Get the corresponding time
        
        esmData=np.matlib.repmat(esmFrame.iloc[beep][esmColumns],esmWindowLength,1)
        combined=esmData
        
        subIndex=[beepTime-hop*t for t in range(esmWindowLength)][::-1]
        for featureFrame in listOfDF:
        
        
            timediff = np.min(np.abs(featureFrame.index-beepTime)) 
            # Find corresponding moment for beep time in the sensor data
            #print(timediff)
            if timediff>timedelta(minutes=esmWindowLength):
                # If corresponding time is too far off, remove beep
                #print("Couldn't find corresponding sensor data")
                continue
            pos=np.argmin(np.abs(featureFrame.index-beepTime))
            # For the smallest time difference find the position in the sensor data
            if pos>esmWindowLength:
                featColumns=featureFrame.keys().tolist() #The names of the features                
                featData=featureFrame.iloc[pos-esmWindowLength:pos][featColumns].values
                # Get corresponding timestamps
                
                # Repeat ESM data for each data point in the window
                combined=np.concatenate((combined,featData),axis=1)
                #Combine ESM & feature data
        if combined.shape[1]==len(combinedColumns):
            esmFeatures=esmFeatures.append(pd.DataFrame(combined,columns=combinedColumns,index=subIndex))
                #Append combined data to the dataframe
    return esmFeatures

In [ ]:
esmFeatures.shape

In [ ]:
#Align them to the ESM File
esmFeatures = alignFeaturesESM([testL,testR,testC],esm[esm['castorID']==110005],esm.keys().tolist(),esmWindowLength=15)

#esmFeatures = esmFeatures.dropna() # Drop all lines with missing values, this definitely needs to be refined for real data

# Save the dataframe
#esmFeatures.to_csv('/Users/jeroenhabets/Desktop/python_code/pilot_testdata/test110001.csv')

In [ ]:
### END OF DATA PREPARATION

In [ ]:
### Simple analysis

# Spearman Correlations
r, p = spearmanr(esmFeatures['sanpar_tremor'],esmFeatures['BandPowerXC']+esmFeatures['BandPowerYC']+esmFeatures['BandPowerZC'])
print('Spearman correlation of r = %f with a p-value of %f' % (r, p))

In [ ]:
### Use Machine Learning to predict some variable from the esm

# Linear Regression
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
# Fitting the linear regression to the data
lr.fit(esmFeatures[['BandPowerXR','BandPowerYL', '#MovementsXL']].values,esmFeatures['stress'])
# Looking at model parameters
# Regression weights
print('Weights are %f, %f, %f ' % (lr.coef_[0], lr.coef_[1], lr.coef_[2]))
# Intercept
print('Intercept is %f' % lr.intercept_)

# Predict target variable based on input data (should be unseen data)
prediction = lr.predict(esmFeatures[['BandPowerXR','BandPowerYL', '#MovementsXL']].values)

In [ ]:
### Data visualization
# Bar plot
fig,ax = plt.subplots()
ax.bar(range(3), lr.coef_,color='r',width=0.4)
ax.bar(np.arange(3)+0.4, lr.coef_+0.01, color='b',width=0.4)
plt.xlabel('Features')
plt.ylabel('Weight')
plt.title('Linear Regression Weights')
plt.xticks(range(3),['BandPowerXR','BandPowerYL', '#MovementsXL'])
plt.yticks(np.arange(0,0.1,0.02))
plt.savefig('weights.png',dpi=60)
plt.show()

In [ ]:
# Scatter Plot
feature1='BandPowerXR'
feature2='BandPowerYL'
fig, ax = plt.subplots(figsize=(7,7))
s1 = ax.scatter(esmFeatures[feature1], esmFeatures[feature2], c=esmFeatures['cheerful'])
plt.xlabel(feature1)
plt.ylabel(feature2)
fig.colorbar(s1,label='Cheerful')
plt.show()

In [ ]:
# Histogram
plt.hist(esmFeatures['#MovementsXR'])

In [ ]:
# Principal Component Analysis
def pca(data):
    # Calculate Covariance Matrix
    cov=np.cov(data.T)
    # Calculate Eigenvalues and Eigenvectors
    w, v = np.linalg.eig(cov)
    # Sort them
    s= np.argsort(w)[::-1]
    return v[:,s]

In [ ]:
# Could also be done in only sensor/esm spaces
esmEigenVectors = pca(esmFeatures.values.astype(float))
transformedFeatureSpace =np.dot(esmEigenVectors[:,:2].T,esmFeatures.values.astype(float).T).T
s1 = plt.scatter(transformedFeatureSpace[:,0],transformedFeatureSpace[:,1],c=esmFeatures['cheerful'])
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.colorbar(s1, label = 'Cheerful')
plt.show()

In [ ]:
## It is very easy to use a more powerful Machine Learning algorithm

from sklearn.ensemble import GradientBoostingRegressor
est = GradientBoostingRegressor()
# Fitting the regression model to our data
est.fit(esmFeatures[['BandPowerXR','BandPowerYL', '#MovementsXL']].values,esmFeatures['stress'])

# Interpreting the model
print(est.feature_importances_)

# Predict target variable based on input data (should be unseen data)
prediction = est.predict(esmFeatures[['BandPowerXR','BandPowerYL', '#MovementsXL']].values)

In [ ]:
from scipy.stats import spearmanr
r,p=spearmanr(alignedFeatures['AccX'].values.tolist(),alignedFeatures['cheerful'].values.tolist(),nan_policy='omit')
print(r,p)

In [ ]:
######### Exploring the sensor data

signal_labels, timeStamps, data, sr = readData(leftWristFile)

#filtData = filter_data(data, sr, 4,8)

In [ ]:
#Extract some sort of feature for all windows and corresponding time stamps
# Peak detection


channel=3
numSamples=data.shape[1]
windowLength=60

win=0
windowData = data[:,win:win+featureWindowLength*sr]

peaks=find_peaks_cwt(windowData[channel,:],np.arange(40,50))

#Maximum for bradykinesia recognition
mMovement = np.max(windowData[3:6,:])
numberOfPeaks = len(peaks)
averageDuration = np.mean(np.diff(peaks))/sr


plt.plot(windowData[channel,:])
plt.scatter(peaks,windowData[channel,peaks],c='r')
plt.show()
print(peaks)

In [ ]:
### Experiments with Orientation estimation
signal_labels, timeStamps, sigbufs, sr = readData(leftWristFile)

In [ ]:
### Transform Gyro data into orientation estimation
from madgwickahrs import MadgwickAHRS
mw = MadgwickAHRS(sampleperiod=1/sr)
euler = np.zeros((3,sigbufs.shape[1]))
for sample in range(sigbufs.shape[1]):
    mw.update_imu(sigbufs[6:,sample],sigbufs[3:6,sample])
    euler[:,sample] = mw.quaternion.to_euler123()

In [ ]:
plt.matshow(euler,aspect='auto')
plt.yticks([0,1,2],['Roll', 'Pitch', 'Yaw'])
plt.xlabel('Time in samples')
plt.show()